# Predicting Diabetes Diagnosis Using Health Factors
Data Science 3870 <br> Dr. Jacob Martin<br>Authors: Abby Howard, Arai Lubas, Ella Stasko



## Introduction
We are interested in exploring what combination of the collected features best predicts whether an individual has diabetes. Given these inputs, we want to determine whether we can create a model to predict this accurately.

Our team will use the Healthcare Diabetes Dataset from Kaggle. Each instance represents various aspects of an individual’s health data and their diagnosis or non-diagnosis of diabetes. It was last updated in 2023, meaning the data is recent and relevant. The dataset is originally from the National Institute of Diabetes and Digestive and Kidney Diseases (NIDDK). There are 2,768 patients included in the data, with 10 features recorded. Each instance represents a unique individual with recorded features listed below:

1.	**Id**: Unique identifier for each data entry.
2. **Pregnancies**: Number of times pregnant. (Includes people who cannot get pregnant).
4.	**Glucose**: Plasma glucose concentration over 2 hours in an oral glucose tolerance test.
5.	**BloodPressure**: Diastolic blood pressure (mm Hg).
6.	**SkinThickness**: Triceps skinfold thickness (mm). Indicator of body fat distribution.
7.	**Insulin**: 2-Hour serum insulin (mu U/ml).
8.	**BMI**: Body mass index (weight in kg / height in m^2).
9.	**DiabetesPedigreeFunction**: Diabetes pedigree function, a genetic score of diabetes.
10.	**Age**: Age in years.
11.	**Outcome**: Binary classification indicating the presence (1) or absence (0) of diabetes.
   
Features we are particularly interested in to predict whether the individual has diabetes are glucose, diabetes pedigree function, and insulin features. We are surprised by the skin thickness and intrigued to determine whether this has any predictive value. 

## Exploratory Data Analysis
Before creating our model to predict diabetes diagnosis, we will evaluate the dataset through visualization. It is important to explore data features to identify any abnormalities in the data. First we need to read in our data and confirm there are no missing values.


In [ ]:
# Import modules
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import warnings 
from scipy.stats import ttest_ind
from sklearn import tree, ensemble
import sklearn.linear_model as lm
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
warnings.filterwarnings('ignore')

In [ ]:
# Read in the dataset and set the patient ID as the index
data = pd.read_csv('Healthcare-Diabetes.csv', index_col= 'Id')
data

In [ ]:
# Check for missing values 
data.isna().any()

In [ ]:
# Check for zero values
(data.iloc[: ,1:8] == 0).sum()

The data does not contain any missing values however, we noticed that there are multiple observations with zero values recorded for features that cannot equal zero (age, BMI, Insulin etc.)

We will assume that for features: Glucose, Blood Pressure, Skin Thickness, Insulin, BMI, Diabetes Pedigree Function, and Age, values of zero are improbable and thus are missing values that need to be removed. We include further analysis to determine how to approach this problem. Features such as Insulin and Skin thickness have a large number of zeros and in order to determine whether these features should be removed from the dataset to preserve sample size or kept at the expense of a smaller sample size, we will perfom individual analysis of each feature against outcome with all zero values removed.

First we create DataFrames of each variable against outcome removing all zero values. We then plot the Age, BMI, and Pregnancies distributions of the data to start gain an understanding of the data sample. 

In [ ]:
# Create DataFrames for each variable against outcome removing zero values to assess trends via violin plots
dataframes = {} 

# Generating individual dataframes for each variable containing the variable and outcome
for col in data:
    if col == 'Pregnancies':
        dataframes[col] = data.loc[:, (col, 'Outcome')]
    if col == 'Outcome':
        pass
    # Remove zero value rows for all variables except pregancies and outcome
    else:
        dataframes[col] = data.loc[:, (col, 'Outcome')].query('`{}` != 0'.format(col))

# Create a variable to hold the desired palette colors for graphs
colors = ['skyblue','palevioletred']

In [ ]:
# Plot the Age Distribution of participants in a subplot
plt.figure(1, figsize = (12,6), clear = True)
plt.tight_layout()
plt.subplot(1,3,1)
sns.histplot(dataframes['Age'], 
             x = 'Age',
             bins = 18,
             color = 'skyblue')
plt.title('Age of Participants in Dataset')

# Plot the BMI distribution of Participants in a sub plot
plt.subplot(1,3,2)
sns.histplot(dataframes['BMI'], 
             x = 'BMI',
             bins = 25,
             color = 'skyblue')
plt.title('BMI of Participants in Dataset')

# Plot the Distribution of the number of pregnancies in a subplot
plt.subplot(1,3,3)
sns.histplot(dataframes['Pregnancies'], 
              x = 'Pregnancies',
              bins = 18,
              color = 'skyblue')
plt.title('Number of Pregnancies of Participants in Dataset')

plt.tight_layout();

From this we see the overall age of the sample is rightly skewed, the overall BMI of the population is normally distributed with a slight right skew and the number of pregnancies in the data is rightly skewed. 

We next isolated each feature and grouped the values by outcome. This allows us to visualize and potentially identify any obvious intra-variable distribution difference against the outcome, diabetes. 

In [ ]:
# Create figure
plt.figure(1, figsize = (8,6), clear = True)
subplot_num = 1

# Loop through features and plot separated by outcome
for key in dataframes:
    plt.subplot(2, 4, subplot_num)
    plt.title(f"{key}", size = 10)
    sns.violinplot(data = dataframes[key], 
                   y = key, 
                   x = 'Outcome', 
                   alpha = 0.8,
                   palette =  ['skyblue', 'skyblue']
                  )
    plt.xlabel('Diabetes')
    plt.xticks([0,1])
    subplot_num +=1 

plt.tight_layout()

We evaluate each feature by comparing the distribution of its values against outcome 


**Pregnancies**: It appears that patients with diabetes have a wider distribution of number of pregnancies and higher med. This becomes hard to interpret as the data -nor its metadata file / documentation- do not specify sex and thus may include patients unable to have children. This might jepordize the valididity of using this feature. Alternatively, as sex is not a feature of this dataset it may not impact diabetes outcome. Thus, pregnancy impact on diabetes outcome could easily be determine by pregnancy or no pregnancy - where a person with or without the ability to get pregnant, who has not been pregrnant, has the same chance of a positive outcome. Regardless, we will perform further analysis of this feature. 

**Glucose**: Patients with diabetes have on average a higher blood glucose level than those without, this is a possible feature for prediction as the distributions appear distinct from one another.

**Blood Pressure**: Patients with diabetes have a slighly higher median diastolic blood pressure than those without, but overall the distributions appear similar.

**Skin Thickeness**: Patients with diabetes have a slighly higher median skin thickness than those without, but overall the distributions appear similar. This is a feature with a high missing rate (800 values) and the minimal difference in distribution and median begins to suggest removal of this feature would be acceptable. 

**Insulin**: Patients with diabetes have a higher median 2-hour serum insulin than those without. This feature has a high missing rate (1330 values) and the significant difference in median insulin value suggests it might impact the effectiveness of prediction. This further solidifies that more evaluation of the insulin feature is needed before its removal. 

**BMI**: Patients with diabetes have a higher median BMI and patients without diabetes follow a wider distribution while patients with diabetes follow a narrow, normal distribution. 

**Diabetes Pedigree Function**: Patients with and without diabetes follow a similar right skewed distribution of diabetes pedigree function with comparable median values.

**Age**: Patients with diabetes are, on average, older than those without and patients with diabetes have a much wider distribution than those without.



### Summary Statistics

To further examine the relationships between the variables and diabetes diagnoses, we performed two-sample independent t-tests to evaluate the differences in means between the groups and whether or not they are statistically significant. 

In [ ]:
# Store the numerical feature names to test
features = ['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin', 'BMI', 'DiabetesPedigreeFunction','Age']

# Store t-test results
t_test_results = []

# Loop through each feature to perform the t-tests
for feature in features:
    group_negative = dataframes[feature][dataframes[feature]['Outcome'] == 0][feature]
    group_positive = dataframes[feature][dataframes[feature]['Outcome'] == 1][feature]
    mean_diff = np.abs(group_negative.mean()-group_positive.mean())

    # Perform t-test
    t_stat, p_val = ttest_ind(group_negative, group_positive, axis = 0, equal_var = False)

    # Store results
    t_test_results.append({
        'Feature': feature,
        'Absolute Mean Difference': mean_diff,
        'T-Statistics': round(t_stat, 3),
        'P-value': p_val,
        'Significant': p_val < 0.05
    })
t_test_df = pd.DataFrame(t_test_results)
t_test_df

All features have a significant p-value. While only the means of Glucose and Insulin are drastically different, because the sample sizes are all large it creates large test statistics and small p-values. P-Values are often ignored in clinical work because it is more clinically relevant to look at the magnitude of the difference of the relevant metric. With a large enough sample size even small changes that are not clinically relevant could be statistially significant. As seen above the mean difference of Blood Pressure between patients with and without diabetes is ~4.6; while this is statistically significant, a difference in Blood Pressure of 4.6mmHg is not clinically relevant. However we can not anticipate which features will be the  useful in predicting diabetes using this information. Yet, Glucose and Insulin have a more relevant magnitude of difference in their mean values across outcomes indicating a higher chance of them being a useful predictor and the t-test analysis does not suggest the removal of any features. 

### Insulin Sub-Analysis

While Insulin has a large percent of its values missing, because of its high predictive potential, we will perform a sub-analysis including this feature. We first create a model including this feature and removing all observations where Insulin has a value of 0. 

and then create a model removing insulin the feature to maintain a larger sample size. We do this to confirm which method will produce a better performing model. 

To maintain our large sample size, we decided to remove the features from the dataset missing a significant number of values, these features are: Insulin and Skin Thickness. We then removed individual observations with remaining missing values from Glucose, Blood Pressure and BMI. However given Insulin's predictive potential we will perform a sub-analysis including insulin with a smaller sample size.

In [ ]:
# Remove observations with missing values of Glucose, Blood Pressure, BME, and Insulin
cleaned_data_insulin = data.query('Glucose !=0 & BloodPressure !=0 & BMI !=0 & Insulin !=0').copy()

# Remove feature from dataset with substantial missing value: Skin Thickness
cleaned_data_insulin.drop(['SkinThickness'], axis = 1, inplace = True)
cleaned_data_insulin

Clearly, maintaining insulin as a feature reduces the sample size of the dataset by almost 50%, (2768 original -> 1339). 

We will next show the pairwise correlation between all features of the cleaned data set with insulin to see if there is any clear relationship between outcome and any two features of the dataset. Here we are mainly looking to identify relationships between insulin and the remaining features. In the general analysis we will further discuss the remaining features.

In [ ]:
# Create correlation matrix
matrix = sns.pairplot(
    cleaned_data_insulin,
    hue = 'Outcome',
    palette = colors,
    plot_kws = {'alpha' : 0.5,
               's' : 10}
)

# Format matrix
matrix._legend.set_title('Diabetes \nOutcome', prop={'size': 14})
matrix._legend.set_bbox_to_anchor((1, 0.9))
plt.suptitle("Correlation Matrix: Data Features Against Diabetes Outcome (0, 1)", y = 1.01, fontsize = 15);

The correlation matrix shows little relationship between insulin and another other feature besides glucose. In this plot we can see a slight positive linear relationship.

We will now split the cleaned data with insulin into training and testing datasets to create a logistic regression model. 

We see a slightly linear, positive relationship between glucose and insuin, however aisde from this there doesn't seem to be any strong correlations between features indicating we aren't at risk of having colinearities in the data. However, there also doesn't seem to be any strong clustering of the data by outcome across any two features, but there does appear to be a strong relationship between an increase and glucose and diabetes outcome across all variables. 

In [ ]:
# Create and fit a logistic regression model 
log_reg_insulin = (
    lm.LogisticRegression(penalty = None,  # No regularization
                          solver = 'lbfgs',
                          max_iter = 1000)
    .fit(X = x_train_insulin, y = y_train_insulin)         # Fit the model to training data
)
# Displaying the model estimates
log_df_insulin = pd.DataFrame({
    'Term': ['Intercept'] + x_train_insulin.columns.tolist(),
    'Estimate': np.concatenate((log_reg_insulin.intercept_, log_reg_insulin.coef_.flatten()), axis = 0).round(3)
})
log_df_insulin['Odds Ratio'] = np.exp(log_df_insulin['Estimate']).round(3)
log_df_insulin

The odds ratio will tell us how the odds of a diabetes diagnosis change with an increase in the predictor feature, while all other features remain constant. In this analysis, ...

Now that we know how impactful each feature is on the model, lets see how accurate the logistic regression model is. 

In [ ]:
# Create a function to convert predicted probabilities to binary predictors
def threshold_pred(p, t):
    return (p >= t) * 1

# Predict probabilities for diabetes and convert them to binary predictions
log_reg_prob_insulin = log_reg_insulin.predict_proba(X = x_test_insulin)[:,1]

# Tune threshold
log_reg_predict_insulin = {}
log_reg_accuracy_insulin = {}
values = np.arange(0.1,1,0.1)
for value in values:
    v = value.round(2)
    log_reg_predict_insulin[f'{v}'] = threshold_pred(log_reg_prob_insulin, v)
    log_reg_accuracy_insulin[f'{v}'] = np.mean(log_reg_predict_insulin[str(v)] == np.array(y_test_insulin)).round(4)

# Print the accuracy of the logistic regression model 
for key in log_reg_accuracy_insulin:
    print(f'The accuracy of model with threshold {key} is: {log_reg_accuracy_insulin[key]}') 
    
print(f'\nThe accuracy of predicting every patient as non-diabetic: {1-np.mean(y_test_insulin).round(4)}') 

# Print the threshold that produces the maximum accuracy
max_key_insulin = max(log_reg_accuracy_insulin, key=log_reg_accuracy_insulin.get)
print(f'The threshold value which produces the model with the maximum accuracy is {max_key_insulin} with an accuracy of {log_reg_accuracy_insulin[max_key_insulin]}')

From our calculations, we can see our logistc regression model has decent accuracy, and will predict a patient's diabetes diagnosis 81.12% of the time. This is better than just predicting every patient was not diabetic, which is correct 71.33% of the time.

To better understand the accuracy of our model we can create a confusion matrix to understand how often the model is correct, and how often it predicts false positives and false negatives.

In [ ]:
# Create a confusion matrix with the percent accurately predicted 
logistic_regression_confusion_matrix_insulin = confusion_matrix(
    y_true = y_test_insulin,
    y_pred = log_reg_predict_insulin[max_key_insulin], 
    normalize = 'true'
)

# Display the confusion matrix
sns.heatmap(logistic_regression_confusion_matrix_insulin,
           annot = True,
           fmt = '.3f',
           cmap = 'Blues',
           annot_kws = {'size': 16},
           xticklabels = ['Non Diabetic','Diabetic'],
           yticklabels = ['Non Diabetic','Diabetic'])

plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix for Logistic Regression");

From the confusion matrix we can see that the model correctly predicted non diabetic patients 85.6% of the time. However, it only correctly predicted patients with diabetes 69.9% of the time. Now, we will determine whether removing insulin as a feature improves the accuracy of the logistic regression model. 

### Complete Analysis

Create cleaned dataset removing observations with missing values for Glucose, Blood Pressure and BMI and remove feature: Insulin and Skin Thickness.

In [ ]:
# Remove observations with missing values of Glucose, Blood Pressure, and BMI
cleaned_data = data.query('Glucose !=0 & BloodPressure !=0 & BMI !=0').copy()

# Remove features from working dataset with substantial missing values: Insulin, Skin Thickness
cleaned_data.drop(['Insulin','SkinThickness'], axis = 1, inplace = True)
cleaned_data

We will show the pairwise correlation between all features of the cleaned data set to see if there is any clear relationship between outcome and any two features of the dataset.

In [ ]:
# Create correlation matrix
matrix = sns.pairplot(
    cleaned_data,
    hue = 'Outcome',
    palette = colors,
    plot_kws = {'alpha' : 0.5,
               's' : 10}
)

# Format matrix
matrix._legend.set_title('Diabetes \nOutcome', prop={'size': 14})
matrix._legend.set_bbox_to_anchor((1, 0.9))
plt.suptitle("Correlation Matrix: Data Features Against Diabetes Outcome (0, 1)", y = 1.01, fontsize = 15);

When looking at the scatter plots we see the same trends as in the previous correlation matrix. We include this again without the inclusion of insulin to highlight the relationships used in the complete analysis.

At the end of our exploratory data analysis, we have satesafactorily cleaned the dataset and can move on to build and evaluate potential models

## Model Building

Prior to building any models, we will split the dataset into testing and training sets. This allows us to determine the accuracy of the model against unseen data.

In [ ]:
# Split the data into train and test datasets 
x_train_insulin, x_test_insulin, y_train_insulin, y_test_insulin = train_test_split(
    cleaned_data_insulin.drop(columns = 'Outcome', axis = 1),
    cleaned_data_insulin.Outcome,
    test_size = 0.3, 
    random_state = 3870)
x_train_insulin

In [ ]:
# Split the data into train and test datasets 
x_train, x_test, y_train, y_test = train_test_split(
    cleaned_data.drop(columns = 'Outcome', axis = 1),
    cleaned_data.Outcome,
    test_size = 0.3, 
    random_state = 3870)
x_train

In [ ]:
# Split the data into train and test datasets 
x_train, x_test, y_train, y_test = train_test_split(
    cleaned_data.drop(columns = 'Outcome', axis = 1),
    cleaned_data.Outcome,
    test_size = 0.3, 
    random_state = 3870)
x_train

### Logistic Regression

We chose to start with a logistic regression model to predict diabetes, utilizing the useful features analyzed in the previous section. Logistic regression is a good starting point for a classification model. 

In [ ]:
# Create and fit a logistic regression model 
log_reg = (
    lm.LogisticRegression(penalty = None,  # No regularization
                          solver = 'lbfgs',
                          max_iter = 1000)
    .fit(X = x_train, y = y_train)         # Fit the model to training data
)
# Displaying the model estimates
log_df = pd.DataFrame({
    'Term': ['Intercept'] + x_train.columns.tolist(),
    'Estimate': np.concatenate((log_reg.intercept_, log_reg.coef_.flatten()), axis = 0).round(3)
})
log_df['Odds Ratio'] = np.exp(log_df['Estimate']).round(3)
log_df

The odds ratio will tell us how the odds of a diabetes diagnosis change with an increase in the predictor feature, while all other features remain constant. In this analysis, we can see that there is little to no association between blood pressure and a diabetes diagnosis, as its odds ratio is close to 1. In contrast, the Diabetes Pedigree Function shows a stronger association, with a higher odds ratio indicating increased odds of a diabetes diagnosis as this value rises. The other features examined, such as BMI, Glucose, and Pregnancies, demonstrate slight associations with diabetes diagnosis, reflected by odds ratios slightly above 1.

Now that we know how impactful each feature is on the model, lets see how accurate the logistic regression model is. 

In [ ]:
# Predict probabilities for diabetes and convert them to binary predictions
log_reg_prob = log_reg.predict_proba(X = x_test)[:,1]

# Tune threshold
log_reg_predict = {}
log_reg_accuracy = {}
values = np.arange(0.1,1,0.1)
for value in values:
    v = value.round(2)
    log_reg_predict[f'{v}'] = threshold_pred(log_reg_prob, v)
    log_reg_accuracy[f'{v}'] = np.mean(log_reg_predict[str(v)] == np.array(y_test)).round(4)

# Print the accuracy of the logistic regression model 
for key in log_reg_accuracy:
    print(f'The accuracy of model with threshold {key} is: {log_reg_accuracy[key]}') 
    
print(f'\nThe accuracy of predicting every patient as non-diabetic: {1-np.mean(y_test).round(4)}') 

# Print the threshold that produces the maximum accuracy
max_key = max(log_reg_accuracy, key=log_reg_accuracy.get)
print(f'The threshold value which produces the model with the maximum accuracy is {max_key} with an accuracy of {log_reg_accuracy[max_key]}')

From our calculations, we can see our logistc regression model has decent accuracy, and will predict a patient's diabetes diagnosis 77.3% of the time. This is better than just predicting every patient was not diabetic, which is correct 65.82% of the time.

To better understand the accuracy of our model we can create a confusion matrix to understand how often the model is correct, and how often it predicts false positives and false negatives.

In [ ]:
# Create a confusion matrix with the percent accurately predicted 
logistic_regression_confusion_matrix = confusion_matrix(
    y_true = y_test,
    y_pred = log_reg_predict[max_key], 
    normalize = 'true'
)

# Display the confusion matrix
sns.heatmap(logistic_regression_confusion_matrix,
           annot = True,
           fmt = '.3f',
           cmap = 'Blues',
           annot_kws = {'size': 16},
           xticklabels = ['Non Diabetic','Diabetic'],
           yticklabels = ['Non Diabetic','Diabetic'])

plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix for Logistic Regression");

From the confusion matrix we can see that the model correctly predicted non diabetic patients 90.1% of the time. However, it only correctly predicted patients with diabetes 52.6% of the time. This means our model is missing about half of our diabetic patients, which poses significant issues when considering the impacts of missing a diagnosis of diabetes in a patient and prolonging the time until treatment. 

Considering that our model could be significantly improved in correctly predicting diabetes, we will move towards ensemble methods: Random Forest and XGBoost.

### Random Forest

First we will perform a grid search to identify the best hyperparameter values before proceeding to creating the Random Forest. 

In [ ]:
# Form the grid of values to choose hyperparameter values from 
hps_grid = {
    'n_estimators': [10, 50, 100, 200, 300, 400, 500],
    'max_features': np.arange(1,7, 1).tolist(),
    'max_depth': [1, 2, 10, 20, 50, None]
}


Perform cross validation to select the best hyperparameters

In [ ]:
# 5-fold cv partition
cv5 = KFold(n_splits = 5, shuffle = True, random_state = 3870)

# Create instance of GridSearchCV engine 
hp_grid_search = GridSearchCV(
    ensemble.RandomForestClassifier(random_state = 3870),
    hps_grid,
    cv = cv5,
    scoring = 'accuracy',
    n_jobs = -1
)

In [ ]:
# Fit the training data to the GridSearcCV engine 
hp_grid_search.fit(X = x_train, y = y_train)

In [ ]:
# Print the best hyperparameters
print(f"The best choice for the number of trees is {hp_grid_search.best_params_['n_estimators']}")
print(f"The best choice for the number of features is {hp_grid_search.best_params_['max_features']}")
print(f"The best choice for max depth is {hp_grid_search.best_params_['max_depth']}")
print(f"which has an accuracy of: {hp_grid_search.best_score_: .4f}")

In [ ]:
# Display the overall best model
best_model = hp_grid_search.best_estimator_
best_model

After the grid search, we have found the best Random Forest model includes one hundred trees, each with one feature, and a maximum depth of twenty. This model generates an R^2 value of 0.870 on average across five cross-validation folds. This is considered a good model, with significant improvement in predicting diabetes diagnoses than that generated by the logistic regression.

Now that we have determined the best hyper parameters for our model and fit it to our data to create the best model, we will evaluate the accuracy of the model using our testing set. 

In [ ]:
# Accuracy of best model with train and test sets
rf_training_accuracy = best_model.score(x_train, y_train)
rf_testing_accuracy = best_model.score(x_test, y_test)
print('Training Accuracy: ',round(rf_training_accuracy, 3), ' Testing Accuracy: ',round(rf_testing_accuracy, 3))

The model perfectly classified all cases in the training set, this might suggest overfitting however the model has a 98.5% accuracy on the testing set, indicating it predicts diabetes very well. Next we will examine the confusion matrix for this model. 

In [ ]:
# Determine the predicted y values from the best model
y_pred = best_model.predict(X=x_test)

# Create a confusion matrix with the percent accurately predicted 
random_forest_confusion_matrix = confusion_matrix(
    y_true = y_test,
    y_pred = y_pred, 
    normalize = 'true'
)

# Display the confusion matrix
sns.heatmap(random_forest_confusion_matrix,
           annot = True,
           fmt = '.3f',
           cmap = 'Blues',
           annot_kws = {'size': 16},
           xticklabels = ['Non Diabetic','Diabetic'],
           yticklabels = ['Non Diabetic','Diabetic'])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix for Random Forest");

The Random Forests confusion matrix shows much improvement from the logistic regression model. It correctly predicted non-diabetics 98.6% of the time, and diabetics 98.1% of the time. This shows significant improvement in the model and promise that diabetes diagnosis can be predicted by useful known health factors. The model’s strong accuracy in both categories suggests it could be a valuable tool in early detection and prevention strategies for at-risk populations.


### XGBoost
The second non-parametric model we will use to predict diabetes in patients is XGBoost. First we must determine the best combination of hyper parameters using a grid search in the same way we did earlier but with XGBClassifier this time. 

In [ ]:
# Form the grid of values to choose hyper parameter values from 

xgb_hp_dict = {
    'learning_rate':    [0.01, 0.05, 0.1, 0.2, 0.3], # Each new prediction's contribution to y_hat
    'n_estimators':     [100, 250, 500, 750, 1000],  # Number of trees to make
    'max_depth':        [1, 3, 5, 7],                # Maximum number of splits: 1, 8, 32, 128 leaf nodes   
    'colsample_bytree': [0.4, 0.6, 0.8, 1],          # Number of features to use per tree (40, 60, 80, & 100%)
    'lambda':           [0, 0.5, 1, 2, 5]            # Regularization parameters to try
}

# 5 fold cross-val
cv5 = KFold(n_splits = 5, shuffle = True, random_state = 3870)

# Defining the model:
xgb_class = XGBClassifier(random_state = 3870, early_stopping_rounds = 50)

# Defining the grid search
grid_search = GridSearchCV(
    estimator = xgb_class,
    param_grid = xgb_hp_dict,
    n_jobs = -1,
    cv = cv5    
)

In [ ]:
#Conducting the grid search
gs_results = (
    grid_search
    .fit(
        x_train, y_train,
        eval_set = [(x_test, y_test)],
        verbose = False
    )
)

#define the best parameters
best_hps = grid_search.best_params_
best_hps

The best hyperparameters for the XGBoost model has 60% of features for each tree, a L2 regularization strenght of 2, a learning rate of 0.3, a maximum tree depth of 5, and 500 boosting rounds. 
Next, we will define the best model and fit it with the testing and training data. 

In [ ]:
#define the best model and fit to data
xgb_best_model = (
    XGBClassifier(
        **best_hps,  # Assigns the dictionary values to the parameter keys
        random_state = 3870,
        early_stopping_rounds = 20
    )
    .fit(
        x_train, y_train,
        eval_set = [(x_test, y_test)],
        verbose = False
    )
)

# Calculating the accuracy of the model
accuracy_xgb_tuned = best_model.score(x_test, y_test)
print('Train Accuracy: ',best_model.score(x_train, y_train), ' Test Accuracy: ', accuracy_xgb_tuned.round(3))

Again, as expected, the accuracy of the XGBoost model on the training data is 1.0, and on the training data is 0.985. This indicates the accuracy of the XGBoost model is very good, and is equivalent to that of the Random Forest model. 

To further determine the quality of the model we will make another confusion matrix. 

In [ ]:
# Determine the predicted y values from the best model
xgb_y_pred = best_model.predict(X = x_test)

# Create a normalized confusion matrix of true/ false positives and negatives
XGB_confusion_matrix = confusion_matrix(
    y_true = y_test,
    y_pred = xgb_y_pred, 
    normalize = 'true'
)
# Display the confusion matrix
sns.heatmap(XGB_confusion_matrix,
           annot = True,
           fmt = '.3f',
           cmap = 'Blues',
           annot_kws = {'size': 16},
           xticklabels = ['Non Diabetic','Diabetic'],
           yticklabels = ['Non Diabetic','Diabetic'])
plt.xlabel("Predicted")
plt.ylabel("Actual");

The XGBoost model has the same accuracy, and predictions as the Random Forest model, indicating that with our data, a more complex model does not inherently make a better model for our data. 

# Summary and Conclusions

Our project aims to explore if we can predict diabetes diagnoses with known health data, via multiple machine learnign models. We explored parametric and non parametric methods: logistic regression, Random Forests, and XGBoost. Useful factors from our data includes glucose levels, BMI, age, Diabetes Pedigree Function, and number of pregnancies. A sub-analysis identified a positive association between the number of pregnancies and diabetes diagnosis, as well as an inclusive finding that considering those who have never been pregnant or cannot become pregnant was still informative.

The first modelling method used was logistic regression, a non-parametric method for binary, categorical outcome variables. This model showed decent accuracy, predicting correctly 77.3% of the time. However, upon further analysis, this model only correctly predicted those without diabetes, and had a high false negative rate of 47.4%. This model would miss almost half of diabetic patients, which could have severe consequences in a medical setting when early intervention, treatment, and diagnosis can impact a patient's wellbeing and quality of life. 

The second model used was Random Forests, which improved upon the accuracy of the logistic regression model to 98.5%. Upon further analysis of the confusion matrix, the model had significant improvement with the false negative rates remaining of 1.9%. 

The third model used was XGBoost, which is a more compelx model, however it did not yield a higher accuracy, remaining at 98.5%. The confusion matrix shows a similar trend, with the false positive rate being equivalent to the 1.9% of the Random Forests model. 

Overall, these models demonstrated the potential of using known health features to accurately predict diabetes in patients, showing promise as a tool for early detection and prevention strategies to improve the quality of life for diabetic patients. 

# Limitations and Recommendations

The first major limitation is the amount of missing values in the 2-hour serum Insulin column in our data set. This required us to eliminate it as a potential predictive feature, which could have improved the model. In addition, the pregnancy column does not discriminate between those who cnnot get pregnant with those who can, resulting in an over reporting of zero pregnancies, therefore potentially skewing the relationship between diabetes and pregnancies, which required an additional sub analysis. Lastly, future research should also focus on improving data quality by minimizing missing values for critical features. 

Second, this data set does not include socioeconomic factors, which we know can be highly associated with diabetes in patients. Factors like sex, race, income, and housing status could all impact the predictive ability of a model for diabetes. 

Third, the data set has a high sample size which creates large test statistics and small p-values, which can potentially inflate the statistical significance of the difference of groups in the two sample t-tests. 

In addition, using machine learning as a predictive tool in diagnostic techniques is still heavily debated in the medical field. Our project highlights the limitations of model building, where the potential for missed diagnoses is high. 

Future recommendations include increasing the number of socioeconomic factors recorded for each patient, and explore other clinical study types to best explore the diagnosis. A cohort study would provide more perspective to the features that impact diabetes by providing a timeframe of when a patient was diagnosed with diabetes, and may illude to what features changed leading to that diagnosis

